In [1]:
import pandas as pd
import numpy as np
import os
import json
import seaborn as sns
import is3_broker_rl
import matplotlib.pyplot as plt
from pathlib import Path
import time

In [2]:
import requests


In [10]:
requests.post("http://localhost:8000/wholesale/history?")

<Response [422]>

In [10]:
ANALYSIS_DIR_NAME = Path(os.environ.get("DATA_DIR", "../data/"))
COMMENTS = ""

In [11]:
reward_prefix = "wholesale_reward.csv"

In [12]:
ANALYSIS_DIR = ANALYSIS_DIR_NAME

In [13]:
df_reward = pd.read_csv(os.path.join(ANALYSIS_DIR,reward_prefix))


In [14]:
def string_to_list(self, input_string: str, delimeter=";"):

        splits = input_string.split(delimeter)
        # self._log.info(f"Splits {splits}")
        return_list = []
        for value in splits:
            if value != "":
                return_list.append(float(value))

        return return_list

In [15]:
def calculate_customer_change(df):
    values = ((df["customer_count"] - df["customer_count"].shift(24)).fillna(0)).values
    df.loc[df.index, "customer_change"] = values
    return df

In [16]:
from typing import List
from pydantic import BaseModel

class Observation(BaseModel):
    gameId: str
    timeslot: int
    p_grid_imbalance: List[float] = []
    p_customer_prosumption: List[float] = []
    p_wholesale_price: List[float] = []
    p_cloud_cover: List[float] = []
    p_temperature: List[float] = []
    p_wind_speed: List[float] = []
    cleared_orders_price: List[float] = []
    cleared_orders_energy: List[float] = []
    cleared_trade_price: List[float] = []
    cleared_trade_energy: List[float] = []
    customer_count: int
    customer_count: int
    total_prosumption: float
    market_position: List[float] = []
    hour_of_day: List[float] = []
    day_of_week: List[float] = []

In [20]:

            
start = time.time()
temp_cc = 0
for index, row in df_reward.iterrows():
    obs = json.loads(row["observation"])
    temp_cc = obs.get("customer_count")
    obs = Observation(
        gameId=obs.get("gameId"),
        timeslot=obs.get("timeslot"),
        p_grid_imbalance=obs.get("p_grid_imbalance"),
        p_customer_prosumption=obs.get("p_customer_prosumption"),
        p_wholesale_price=obs.get("p_wholesale_price"),
        p_cloud_cover=obs.get("p_cloud_cover"),
        p_temperature=obs.get("p_temperature"),
        p_wind_speed=obs.get("p_wind_speed"),
        cleared_orders_price=obs.get("cleared_orders_price"),  # Inputs empty values. These will be filled later.
        cleared_orders_energy=obs.get("cleared_orders_energy"),  # Inputs empty values. These will be filled later.
        cleared_trade_price=obs.get("cleared_trade_price"), # Inputs empty values. These will be filled later.
        cleared_trade_energy=obs.get("cleared_trade_energy"),  # Inputs empty values. These will be filled later.
        customer_count=temp_cc,
        total_prosumption=obs.get("total_prosumption"),
        hour_of_day=obs.get("hour_of_day"),
        day_of_week=obs.get("day_of_week"),
        market_position=obs.get("market_position"),
    )
    #obs = is3_broker_rl.api.wholesale_controller._standardize_observation(obs)
    #self._log.info(f"Obs feature: {obs.to_feature_vector()}")
    reward = row["reward"]
    action_str = row["last_action"]
    
    action = string_to_list(action_str)
    df = pd.DataFrame(
            {
                "episode_id": row.episode_id,
                "reward": reward,
                "balancing_reward":row.balancing_reward,
                "wholesale_reward":row.wholesale_reward,
                "tariff_reward":row.tariff_reward,
                "shaped_return": row.shaped_return,
                "observation": obs,
                "last_action": row.action,
                "sum_mWh": row.sum_mWh,
            },
            index=[0],
        )
    break
        

TypeError: 'int' object is not subscriptable

In [ ]:
df_reward